In [5]:
#Time given is UTC. 
#This pulls the national actual and forecast emission factors. Intensity has a 14 day limit. So will need to batch over longer periods.

import requests
import pandas as pd
from datetime import datetime, timedelta

# === CONFIGURATION ===
start_date_str = "2024-06-30T23:00Z"
end_date_str   = "2025-06-30T23:00Z"  # example range > 14 days

folder = r"C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data"
file_out = fr"{folder}\carbon_intensity_with_mix_{start_date_str[:10]}_to_{end_date_str[:10]}.csv"

headers = {"Accept": "application/json"}
max_window = timedelta(days=13, hours=23, minutes=30)

# === DATE PARSING ===
def parse_utc(datestr):
    return datetime.strptime(datestr, "%Y-%m-%dT%H:%MZ")

start_dt = parse_utc(start_date_str)
end_dt   = parse_utc(end_date_str)

# === STORAGE ===
intensity_rows = []
generation_rows = []

# === LOOP OVER CHUNKS ===
curr_start = start_dt
while curr_start < end_dt:
    curr_end = min(curr_start + max_window, end_dt)
    from_str = curr_start.strftime("%Y-%m-%dT%H:%MZ")
    to_str   = curr_end.strftime("%Y-%m-%dT%H:%MZ")

    print(f"Fetching chunk: {from_str} to {to_str}")

    # === INTENSITY API ===
    int_url = f"https://api.carbonintensity.org.uk/intensity/{from_str}/{to_str}"
    int_resp = requests.get(int_url, headers=headers)
    int_resp.raise_for_status()
    for item in int_resp.json()["data"]:
        intensity_rows.append({
            "from": item["from"],
            "to": item["to"],
            "forecast": item["intensity"].get("forecast"),
            "actual": item["intensity"].get("actual"),
            "index": item["intensity"].get("index")
        })

    # === GENERATION MIX API ===
    mix_url = f"https://api.carbonintensity.org.uk/generation/{from_str}/{to_str}"
    mix_resp = requests.get(mix_url, headers=headers)
    mix_resp.raise_for_status()
    for item in mix_resp.json()["data"]:
        row = {"from": item["from"], "to": item["to"]}
        for fuel in item["generationmix"]:
            row[fuel["fuel"]] = fuel["perc"]
        generation_rows.append(row)

    curr_start = curr_end

# === CONVERT TO DATAFRAMES ===
intensity_df = pd.DataFrame(intensity_rows).drop_duplicates(subset=["from", "to"])
generation_df = pd.DataFrame(generation_rows).drop_duplicates(subset=["from", "to"])

# === MERGE AND SAVE ===
merged_df = intensity_df.merge(generation_df, on=["from", "to"], how="left")
merged_df.to_csv(file_out, index=False)
print(f"Saved combined CSV to: {file_out}")


Fetching chunk: 2024-06-30T23:00Z to 2024-07-14T22:30Z
Fetching chunk: 2024-07-14T22:30Z to 2024-07-28T22:00Z
Fetching chunk: 2024-07-28T22:00Z to 2024-08-11T21:30Z
Fetching chunk: 2024-08-11T21:30Z to 2024-08-25T21:00Z
Fetching chunk: 2024-08-25T21:00Z to 2024-09-08T20:30Z
Fetching chunk: 2024-09-08T20:30Z to 2024-09-22T20:00Z
Fetching chunk: 2024-09-22T20:00Z to 2024-10-06T19:30Z
Fetching chunk: 2024-10-06T19:30Z to 2024-10-20T19:00Z
Fetching chunk: 2024-10-20T19:00Z to 2024-11-03T18:30Z
Fetching chunk: 2024-11-03T18:30Z to 2024-11-17T18:00Z
Fetching chunk: 2024-11-17T18:00Z to 2024-12-01T17:30Z
Fetching chunk: 2024-12-01T17:30Z to 2024-12-15T17:00Z
Fetching chunk: 2024-12-15T17:00Z to 2024-12-29T16:30Z
Fetching chunk: 2024-12-29T16:30Z to 2025-01-12T16:00Z
Fetching chunk: 2025-01-12T16:00Z to 2025-01-26T15:30Z
Fetching chunk: 2025-01-26T15:30Z to 2025-02-09T15:00Z
Fetching chunk: 2025-02-09T15:00Z to 2025-02-23T14:30Z
Fetching chunk: 2025-02-23T14:30Z to 2025-03-09T14:00Z
Fetching c